In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Chinese Restaurant Process

First customer always chooses the first table.

The $n$th customers afterwards occupy the first _unoccupied_ table
with probability $\frac{\alpha}{n-1+\alpha}$,
and occupies an _already occupied_ table
with probability $\frac{c}{n-1+\alpha}$.

Here:

- $n$ is the index of the customers after the first.
- $c$ is the number of people already sitting at that table.
- $\alpha$ is a parameter of the Chinese Restaurant Process.

## Let's simulate this!

In [ ]:
import jax.numpy as np
from jax import jit
from jax.ops import index_update, index



def create_alpha_vector(alpha, table_assignments, n, current_open_table):
    v = np.zeros_like(table_assignments)
    v = index_update(v, index[current_open_table], alpha)
    return v / (n - 1 + alpha)

In [ ]:
table_assignments = np.zeros(shape=(10,))
table_assignments = index_update(table_assignments, index[0], 1)

create_alpha_vector(5, table_assignments, 2, 1)

In [ ]:

def create_occupied_vector(alpha, table_assignments, n, current_open_table):
    v = table_assignments / (n - 1 + alpha)
    return v

In [ ]:
create_occupied_vector(5, table_assignments, 2, 1)

In [ ]:
table_assignments = np.zeros(shape=(10,))

table_assignments = index_update(table_assignments, index[0], 1)

np.min(np.where(table_assignments == 0)[0])

In [ ]:
from jax.scipy.special import logit
from jax import vmap, lax, jit
from jax.random import categorical, PRNGKey, split

p = np.array([0.1, 0.8, 0.1])
logit_p = np.log(p / (1 -p))
categorical(k, logit_p)

In [ ]:
k = PRNGKey(42)

def one_draw(k, p, zeros):
    logits = logit(p)
    idx = categorical(k, logits)
    draw = index_update(zeros, index[idx], 1)
    return draw

def f(carry, x):
    k, p = carry
    # x is our zeros
    draw = one_draw(k, p, x)
    k, _ = split(k)
    return (k, p), draw

def multinomial(k, n, p):
    n_draws = 10000
    a = np.zeros(shape=(n_draws, len(p)))
    (k, p), draws = lax.scan(f, (k, p), a)
    return np.sum(draws, axis=0)

multinomial(k, n=1000, p=np.array([0.3, 0.3, 0.4]))

In [ ]:
draws = multinomial(k, n=1000, p=np.array([0.3, 0.3, 0.4]))
draws

In [ ]:
import numpy as onp
from tqdm import tqdm

alpha = 3
table_assignments = np.zeros(shape=(alpha * 10,))
table_assignments = index_update(table_assignments, index[0], 1)

current_open_table = np.min(np.where(table_assignments == 0)[0])
n_customers = 1000
for n in tqdm(range(2, n_customers+1)):
    prob_vect = create_alpha_vector(alpha, table_assignments, n, current_open_table) + create_occupied_vector(alpha, table_assignments, n, current_open_table)
    assignment_vect = one_draw(k, prob_vect, np.zeros_like(prob_vect))
    table_assignments = np.squeeze(table_assignments + assignment_vect)
    current_open_table = np.min(np.where(table_assignments == 0)[0])
    k, _ = split(k)
prob_vect

## Stick-breaking process

In [ ]:
import jax
import numpy as onp

# Taken from https://stats.stackexchange.com/questions/396315/coding-a-simple-stick-breaking-process-in-python

def stick_breaking(k, num_weights, alpha):
    k, _ = split(k)
    betas = onp.random.beta(1,alpha, size=(num_weights,)) 
    betas[1:] *= onp.cumprod(1 - betas[:-1])
    return betas


stick_breaking(k, num_weights=max_num_classes, alpha=3).sum()


In [ ]:
import matplotlib.pyplot as plt

k, _ = split(k)
max_num_classes = 30

def stick_breaking_jax(k, num_weights, alpha):
    k, _ = split(k)
    betas = jax.random.beta(k, 1, alpha, shape=(num_weights,))
    betas = index_update(betas, index[1:], np.cumprod(1 - betas[:-1]))
    return betas/betas.sum()

weights = stick_breaking_jax(k, num_weights=max_num_classes, alpha=0.5)
plt.plot(weights)

The $\alpha$ parameter is proportional to the number of components that we end up using.

## Generate mixture gaussian from weights

In [ ]:
from jax.random import categorical
from jax.scipy.special import logit

k, _ = split(k)
n_observations = 300
indices = categorical(k, logit(weights), shape=(n_observations,))
indices

$X \sim N(\mu, \sigma)$ is equivalent to:

$$ \hat{X} \sim N(0, 1) $$
$$ X = \sigma\hat{X} + \mu$$

In [ ]:
from jax.random import normal

mus = np.linspace(0, 350, num=max_num_classes)
sigmas = np.ones(shape=(max_num_classes)) * 2
mus[indices] + sigmas[indices] * normal(k, shape=(n_observations,))

In [ ]:
def dp_mixture_gaussian(k, alpha, max_num_classes, num_observations, mus, sigmas):
    weights = stick_breaking_jax(k, num_weights=max_num_classes, alpha=alpha)
    indices = categorical(k, logit(weights), shape=(n_observations,))
    return mus[indices] + sigmas[indices] * normal(k, shape=(n_observations,))

mus = np.linspace(0, 350, num=max_num_classes)
sigmas = np.ones(shape=(max_num_classes))

draws = dp_mixture_gaussian(k, alpha=4, max_num_classes=45, num_observations=100, mus=mus, sigmas=sigmas)

In [ ]:
draws

In [ ]:
plt.hist(draws)

In [ ]:
def ecdf_scatter(data):
    x, y = np.sort(data), np.arange(1, len(data)+1) / len(data)
    plt.scatter(x, y)
    plt.show()
    
ecdf_scatter(draws)